In [24]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

import os

# Définir la variable d'environnement JAVA_HOME
os.environ['JAVA_HOME'] = r"C:\Users\Zakaria-Laptop\AppData\Local\Programs\Eclipse Adoptium\jdk-21.0.5.11-hotspot"

In [2]:
# spark session
spark = SparkSession.builder.appName("PySparkExercise").getOrCreate()

In [28]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("foridurrahman/finalcial-analysis-quantitative-data")

print("Path to dataset files:", path)

100%|██████████| 84.0k/84.0k [00:00<00:00, 431kB/s]

Extracting files...
Path to dataset files: C:\Users\Zakaria-Laptop\.cache\kagglehub\datasets\foridurrahman\finalcial-analysis-quantitative-data\versions\1


In [124]:
file_path = "C:\\Users\\Zakaria-Laptop\\.cache\\kagglehub\\datasets\\foridurrahman\\finalcial-analysis-quantitative-data\\versions\\1\\finalcial analysis quantitative data.csv"

# read data
financial_df = spark.read.csv(file_path, header=True, inferSchema=True, sep=";")
financial_df.show(5)

+----------+-------+-----------+---------------+----------+---------------------+------------+-------------+-----------+------------+------------+------------+----------+------------+------------+----+
|   Segment|Country|   Product | Discount Band |Units Sold| Manufacturing Price | Sale Price | Gross Sales | Discounts |      Sales |       COGS |     Profit |      Date|Month Number| Month Name |Year|
+----------+-------+-----------+---------------+----------+---------------------+------------+-------------+-----------+------------+------------+------------+----------+------------+------------+----+
|Government| Canada| Carretera |          None |    1618,5|               $3,00 |     $20,00 |  $32 370,00 |      $-   | $32 370,00 | $16 185,00 | $16 185,00 |01/01/2014|           1|    January |2014|
|Government|Germany| Carretera |          None |      1321|               $3,00 |     $20,00 |  $26 420,00 |      $-   | $26 420,00 | $13 210,00 | $13 210,00 |01/01/2014|           1|    Janua

In [125]:
# Renommer les colonnes en enlevant les espaces
new_columns = [col.strip() for col in financial_df.columns]
financial_df = financial_df.toDF(*new_columns)

financial_df.show(3)


+----------+-------+-----------+-------------+----------+-------------------+----------+------------+---------+------------+------------+------------+----------+------------+----------+----+
|   Segment|Country|    Product|Discount Band|Units Sold|Manufacturing Price|Sale Price| Gross Sales|Discounts|       Sales|        COGS|      Profit|      Date|Month Number|Month Name|Year|
+----------+-------+-----------+-------------+----------+-------------------+----------+------------+---------+------------+------------+------------+----------+------------+----------+----+
|Government| Canada| Carretera |        None |    1618,5|             $3,00 |   $20,00 | $32 370,00 |    $-   | $32 370,00 | $16 185,00 | $16 185,00 |01/01/2014|           1|  January |2014|
|Government|Germany| Carretera |        None |      1321|             $3,00 |   $20,00 | $26 420,00 |    $-   | $26 420,00 | $13 210,00 | $13 210,00 |01/01/2014|           1|  January |2014|
| Midmarket| France| Carretera |        None 

In [128]:
from pyspark.sql import functions as F

def transform_column_to_float(col_name):
    return F.regexp_replace(F.regexp_replace(F.regexp_replace(F.col(col_name), r'\$', ''), r'\s', ''), ',', '.').cast('float')

columns_to_float = ['Units Sold', 'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Sales', 'COGS', 'Profit']

for column in columns_to_float:
    financial_df = financial_df.withColumn(column, transform_column_to_float(column))

financial_df.show(5)


+----------+-------+-----------+-------------+----------+-------------------+----------+-----------+---------+-------+-------+-------+----------+------------+----------+----+
|   Segment|Country|    Product|Discount Band|Units Sold|Manufacturing Price|Sale Price|Gross Sales|Discounts|  Sales|   COGS| Profit|      Date|Month Number|Month Name|Year|
+----------+-------+-----------+-------------+----------+-------------------+----------+-----------+---------+-------+-------+-------+----------+------------+----------+----+
|Government| Canada| Carretera |        None |    1618.5|                3.0|      20.0|    32370.0|    $-   |32370.0|16185.0|16185.0|01/01/2014|           1|  January |2014|
|Government|Germany| Carretera |        None |    1321.0|                3.0|      20.0|    26420.0|    $-   |26420.0|13210.0|13210.0|01/01/2014|           1|  January |2014|
| Midmarket| France| Carretera |        None |    2178.0|                3.0|      15.0|    32670.0|    $-   |32670.0|21780.0

Question : Quel est le total des ventes (Sales) par segment (Segment) ?